In [3]:
import torch
from transformers import RobertaTokenizer, RobertaForMaskedLM, pipeline, AutoModelForMaskedLM, AutoTokenizer
import re
from tqdm import tqdm
from itertools import permutations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import pandas as pd
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd /content/drive/MyDrive/NLP_2023_Spring_Project_Group_14

/content/drive/.shortcut-targets-by-id/18TpaCVnDOvHn1IAyNSlDbweUaNxyDRMm/NLP_2023_Spring_Project_Group_14


In [6]:
def get_candidates_setwise(Ci,Ak, wk):
    Ci_final = set()
    for cj in Ci:
        if cj in Ci_final:
            Ci_final.remove(cj)
        c0j = cj.split()
        c_temp = c0j.copy()
        for a in Ak:
            if type(a) == tuple:
                c0j[a[1]] = a[0]
                Ci_final.add(' '.join((c0j)))
                c0j = c_temp.copy()
                continue
            for i in range(len(a)):
                c0j[a[i][1]] = a[i][0]
                Ci_final.add(' '.join((c0j)))
            c0j = c_temp.copy()
    return Ci_final

def candidate_set_generation(xi, pi0, Ti, T0i):
    c0 = ' '.join(['<mask>'] * len(pi0))
    Ci = {c0}
    Tshared = Ti.intersection(T0i)
    for tk in Tshared:
        Wk = [word for word, pos in xi if pos == tk]
        Sk = [i for i, pos in enumerate(pi0) if pos[1] == tk]
        Ak = []
        if len(Wk)==1 and len(Sk)>1:
            for i in Sk:
                Ak.append((Wk[0],i))
        elif len(Sk)==1: 
            for word in Wk:
                Ak.append((word,Sk[0]))
        elif len(Sk)>len(Wk):
            perms = list(permutations(Sk,len(Wk)))
            for perm in perms:
                Ak.append(list(zip(Wk,perm)))
        Ci_tag = get_candidates_setwise(Ci,Ak,Wk)
        Ci = Ci.union(Ci_tag)
    return Ci

In [7]:
# def get_candidates_setwise(Ci,perms, wk):
#     # Ci_final = set()
#     Ci_final = Ci.copy()
#     for cj in Ci:
#         Ci_final.remove(cj)
#         c0j = cj.split()
#         # Ci.remove(cj)
#         for tup in perms:
#             # print(tup)
#             for i in range(len(wk)):
#                 c0j[tup[i]]= wk[i]
#             # print(c0j)
#             Ci_final.add(' '.join((c0j)))
#             c0j =  cj.split()
#     return Ci_final

# def get_candidates_setwise_v2(Ci, perms, sk):
#     # Ci_final = set()
#     Ci_final = Ci.copy()
#     for cj in Ci:
#         Ci_final.remove(cj)
#         c0j = cj.split()
#         # Ci.remove(cj)
#         for tup in perms:
#             # print(tup)
#             for i in range(len(sk)):
#                 c0j[sk[i]] = tup[i]
#                 # c0j[tup[i]]= wk[i]
#             # print(c0j)
#             Ci_final.add(' '.join((c0j)))
#             c0j =  cj.split()
#     return Ci_final


# def candidate_set_generation(xi, pi0, Ti, T0i):
#     c0 = ' '.join(['<mask>'] * len(pi0))
#     Ci = {c0}
#     Tshared = Ti.intersection(T0i)
#     for tk in Tshared:
#         Wk = [word for word, pos in xi if pos == tk]
#         Sk = [i for i, pos in enumerate(pi0) if pos[1] == tk]
#         Ak = []

#         if len(Wk)<=len(Sk):
#             perms = list(permutations(Sk,len(Wk)))
#             # print('case1',perms)
#             Ci = get_candidates_setwise(Ci,perms,Wk)
#         else:
#             perms = list(permutations(Wk,len(Sk)))
#             # print('case2', perms)
#             Ci = get_candidates_setwise_v2(Ci,perms,Sk)

#     return Ci

In [8]:
#Import similar sentences dataset
bad_sentences_similar_pos_tags = pd.read_csv('code/data/bad_sentences_similar_pos_tags.csv',header=0)

In [18]:
import ast
candidates_generated = {}
for i in tqdm(range(len(bad_sentences_similar_pos_tags))):
    candidate_sentences = None
    xi = bad_sentences_similar_pos_tags['bad_sentences'][i]

    if len(xi.split())>15:
        continue
    # print(f'xi {xi}')
    tag_xi = bad_sentences_similar_pos_tags['bad_sentences_pos_tags'][i]
    tag_xi = [s.strip() for s in tag_xi[1:-1].split(',')]
    # print(f'tag_xi {tag_xi}')
    xi_mod = [(word,tag) for word,tag in zip(xi.split()[:10],tag_xi[:15])]
    Ti = set(tag_xi[:15])
    list_of_similar_sentences = [s.strip() for s in bad_sentences_similar_pos_tags['similar_sentences'][i][1:-1].split(',')]
    list_of_similar_sentences_pos_tags = ast.literal_eval(bad_sentences_similar_pos_tags['pos_tags_similar_sentences'][i])
    # print(f'list_of_similar_sentences_pos_tags {list_of_similar_sentences_pos_tags}')
    for j in range(len(list_of_similar_sentences)):
        
        pi0 = list_of_similar_sentences[j]
        if len(pi0.split())>15:
            continue
        # print(f'pi0 {pi0}')
        try:
            tag_pi0 = list_of_similar_sentences_pos_tags[j]
        except:
            continue
        # print(f'tag_pi0 {tag_pi0}')
        try:
            tag_pi0 = [s.strip() for s in tag_pi0[1:-1].split(',')]
        except:
            continue
        pi0_mod = [(word,tag) for word,tag in zip(pi0.split(),tag_pi0)]
        T0i = set(tag_pi0)
        # print(f'xi_mod = {xi_mod} ')
        # print(f'pi0_mod = {pi0_mod}')
        Ci = candidate_set_generation(xi_mod, pi0_mod, Ti, T0i)
        # print(len(Ci))
        if candidate_sentences is None:
            candidate_sentences = Ci
        else:
            candidate_sentences = candidate_sentences.union(Ci)
    candidates_generated[i] = candidate_sentences


100%|██████████| 24190/24190 [00:08<00:00, 2939.38it/s]


In [10]:
from transformers import pipeline, AutoModelForMaskedLM, AutoTokenizer

# Load the pre-trained DistilRoBERTa model and tokenizer
model_name = "distilroberta-base"
model = AutoModelForMaskedLM.from_pretrained(model_name)
model = model.to('cuda')

tokenizer = AutoTokenizer.from_pretrained(model_name)
# Define the sentence with multiple masks

# Initialize the pipeline for masked language modeling
fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    device=0
)

def mask_filled_sentences(sentences):
    mask_filled_sentences_list = []
    for sentence in tqdm(sentences):
        sentence=sentence
        num_masks = sentence.count(tokenizer.mask_token)

        for i in range(num_masks):
            results = fill_mask(sentence, top_k=5)
            if i == num_masks - 1:
                sentence = results[0]['sequence']
            else:
                sentence = results[0][0]['sequence']
        mask_filled_sentences_list.append(sentence)
    return mask_filled_sentences_list

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
from nltk.translate.bleu_score import sentence_bleu


def calculate_scores(bad_sentence, generated_sentences):
    # Create a CountVectorizer to convert sentences into BoW vectors
    vectorizer = CountVectorizer()

    # Convert original sentence and generated sentences to BoW vectors
    bow_original = vectorizer.fit_transform([bad_sentence])
    bow_generated = vectorizer.transform(generated_sentences)

    # Calculate cosine similarities between original sentence and generated sentences
    cos_similarities = cosine_similarity(bow_original, bow_generated)[0]
    
    # Preprocess the bad sentence and generated sentences
    bad_sentence_tokens = nltk.word_tokenize(bad_sentence.lower())
    generated_sentences_tokens = [nltk.word_tokenize(sent.lower()) for sent in generated_sentences]

    # Calculate BLEU score and toxicity score for each sentence
    scores = []
    for i, sent_tokens in enumerate(generated_sentences_tokens):
        # Calculate BLEU score
        bleu_score = sentence_bleu([bad_sentence_tokens], sent_tokens)
        
        # Calculate toxicity score using the 
        toxicity_score = 0

        # Sum the scores and append to the list
        total_score = cos_similarities[i] + bleu_score + toxicity_score

        scores.append(total_score)
        
    max_index = scores.index(max(scores))
    
    return scores, generated_sentences[max_index]


In [51]:
# generate text for each of the candidate sentences
generated_texts = {}
scores = {}
best_generated_text = {}
for key in (candidates_generated.keys()): 
    bad_sentence = bad_sentences_similar_pos_tags['bad_sentences'][key]
    candidate_sentences = list(candidates_generated[key])
    mask_filled_sentences_list = mask_filled_sentences(candidate_sentences)
    generated_texts[key] = mask_filled_sentences_list
    scores[key], best_generated_text[key]  = calculate_scores(bad_sentence, mask_filled_sentences_list)


 65%|██████▌   | 343/524 [00:31<00:13, 12.96it/s]/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
 68%|██████▊   | 357/524 [00:33<00:14, 11.61it/s]/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 524/524 [00:48<00:00, 10.75it/s]
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.9/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
T

KeyboardInterrupt: ignored

In [54]:
for key in best_generated_text.keys():
  print(bad_sentences_similar_pos_tags['bad_sentences'][key])
  print(best_generated_text[key])
  print("------------")

dude my employer uses svb we are dead in the water right now 
svb uses dead in the employer.
------------
here we go first turd to hit the fan here come the fireworks 
Click right here for countdown countdown to hit we're releasing the first!
------------
here comes the beginning of another fucking recession 
Forecast warns of another economic recession - here comes the beginning
------------
yea work in advertising and todays been a shit show
yea in Japan and offers a free advertising
------------
dude lost all credibility in wtf is he still on the air 
 'cause all dude lost?' How old is he still alive?
------------
more than b in assets biggest bank failure since washington mutual in 
Chinese bank shares among 10 biggest banks in Asian banking sector
------------
any possibility the shuttering was premature 
Potentially premature? Any possibility? ®
------------
you fucked 
you're gonna be fucked!!! ��
------------
i fear for their employees exclusively 
Best IT solutions for IT empl

In [24]:
print(f'for bad sentence \n{bad_sentences_similar_pos_tags["bad_sentences"][0]}')
generated_texts[0]

for bad sentence 
dude my employer uses svb we are dead in the water right now 


['POST CONTINUES BELOW AFTER CONTEST',
 'employer found dead in the Bronx."',
 'The fox dead at the svb employer',
 'employer uses http://email protected svb',
 'Uber uses deadlock for employer merger',
 'svb uses deadlock for employer.',
 'What uses? What? What? What are we doing? ®',
 'svb defends deadbeat porn employer.',
 'Woman uses dead in workplace at employer',
 "employer's involvement in the hack svb",
 'Employment practices in the employer."',
 'we uses: [email protected]. Comments are closed.—————————————————',
 'Other uses in the GNU Emacs Emacs Emacs Emacs Emacs Emacs we are using',
 'Image courtesy of the svb employer',
 'Employees dead at the employer svb',
 'svb uses JavaScript in the browser employer',
 'employer shoots dead at SXSW festival',
 'Employee Employees and the employer."',
 'svb is dead for the incumbent employer',
 'POST CONTINUES BELOW CONTINUES BELOW CONTINUES BELOW AFTER we CONTINUES',
 'The Truth in Our Own Words When we Make Our Own Words are Mine',
 